## Работа с данными

In [1]:
import os
import csv
import pandas as pd
from langdetect import detect
import re

Сравниваю датасет openalex, скаченный и почищенный Г.А.Морозом, и свой датасет, собранный через скрипт R, выбираем оттуда удаленные аннотации

In [17]:
directory = "openalex"
direct_new = 'openalex_new'
old = os.listdir(directory)
new = os.listdir(direct_new)
rows_to_move = []
for d in old:
    if d in new:
        file = os.listdir(f'openalex/{d}')
        file_new = os.listdir(f'openalex_new/{d}')
        with open(f'openalex/{d}/{file[0]}', 'r') as csv_file:
            csv_reader = csv.DictReader(csv_file)
            for row in csv_reader:
                if not row['abstract']:          # если ячейка аннотации пуста, значит она была некачественная
                    try:
                        rows_to_move.append(row['id'])
                    except KeyError:
                        break
print(len(rows_to_move))

132433


In [3]:
print(rows_to_move[:50])

['https://openalex.org/W2976748684', 'https://openalex.org/W2975215321', 'https://openalex.org/W2976407419', 'https://openalex.org/W2996282745', 'https://openalex.org/W2994820132', 'https://openalex.org/W2952743258', 'https://openalex.org/W2976313432', 'https://openalex.org/W4244468353', 'https://openalex.org/W4280571655', 'https://openalex.org/W2953175165', 'https://openalex.org/W2975075108', 'https://openalex.org/W2996519368', 'https://openalex.org/W3048338282', 'https://openalex.org/W3084874087', 'https://openalex.org/W3093761849', 'https://openalex.org/W3121372271', 'https://openalex.org/W3122277459', 'https://openalex.org/W3182255001', 'https://openalex.org/W3202005375', 'https://openalex.org/W4200461446', 'https://openalex.org/W4205324276', 'https://openalex.org/W4206945945', 'https://openalex.org/W4220791381', 'https://openalex.org/W4220870651', 'https://openalex.org/W4226354590', 'https://openalex.org/W4226409572', 'https://openalex.org/W4236836299', 'https://openalex.org/W4239

Записываем их в отдельный датасет.

In [21]:
directory = "openalex"
direct_new = 'openalex_new'
old = os.listdir(directory)
new = os.listdir(direct_new)
for d in new:
    if d in old:
        file_new = os.listdir(f'openalex_new/{d}')
        with open(f'openalex_new/{d}/{file_new[0]}', 'r') as csv2_file:
            csv2_reader = csv.DictReader(csv2_file)
            with open('output_del.csv', 'a', newline='') as out_file:
                fieldnames = csv2_reader.fieldnames
                csv_writer = csv.DictWriter(out_file, fieldnames=fieldnames)
                csv_writer.writeheader()
                for row in csv2_reader:
                    if row['id'] in rows_to_move:
                        csv_writer.writerow(row)

Теперь сравниваем датасеты и выбираем уже новые аннотации, которые появились

In [5]:
directory = "openalex"
direct_new = 'openalex_new'
old = os.listdir(directory)
new = os.listdir(direct_new)
for d in new:
    if d not in old:
        print(d)

CONCENTRIC
ESTUDIOS DE FONETICA EXPERIMENTAL
HRVATSKI DIJALEKTOLOSKI ZBORNIK
JOURNAL OF THE INTERNATIONAL COUNCIL OF ONOMASTIC SCIENCE
QUADERNS DE FILOLOGIA
ZEITSCHRIFT FГњR DIALEKTOLOGIE UND LINGUISTIK
ZEITSCHRIFT FГњR GERMANISTISCHE LINGUISTIK


In [6]:
new_ids = []
for d in old:
    if d in new:
        file_new = os.listdir(f'openalex_new/{d}')
        file = os.listdir(f'openalex/{d}')
        df1 = pd.read_csv(f'openalex_new/{d}/{file_new[0]}')
        df2 = pd.read_csv(f'openalex/{d}/{file[0]}')
        try:
            for id_value in df1['id']:
    # Проверяем, есть ли текущий id в колонке id второго файла
                if id_value not in df2['id'].values:
        # Если id не найден, добавляем его в список new_ids
                    new_ids.append(id_value)
        except KeyError:
            continue
print(len(new_ids))
print(new_ids[:30])

8337
['https://openalex.org/W4377983438', 'https://openalex.org/W4379520738', 'https://openalex.org/W4379520763', 'https://openalex.org/W4380323563', 'https://openalex.org/W4380481457', 'https://openalex.org/W4385461748', 'https://openalex.org/W4385719547', 'https://openalex.org/W4385724970', 'https://openalex.org/W4385725340', 'https://openalex.org/W4385952441', 'https://openalex.org/W4386161502', 'https://openalex.org/W4389307715', 'https://openalex.org/W4389308171', 'https://openalex.org/W4386526679', 'https://openalex.org/W4386835821', 'https://openalex.org/W4389608090', 'https://openalex.org/W4389608091', 'https://openalex.org/W4380488118', 'https://openalex.org/W4384926823', 'https://openalex.org/W4376874244', 'https://openalex.org/W4377693550', 'https://openalex.org/W4380885877', 'https://openalex.org/W4381382067', 'https://openalex.org/W4381435041', 'https://openalex.org/W4386768603', 'https://openalex.org/W4386820177', 'https://openalex.org/W4387531215', 'https://openalex.org/

Записываем новые аннотации также в отдельный файл

In [22]:
new = os.listdir('openalex_new')
with open('output_new.csv', 'a', newline='') as outputnew_file:
    fieldnames = csv_reader.fieldnames
    csv_writer = csv.DictWriter(outputnew_file, fieldnames=fieldnames)
    csv_writer.writeheader()
    for d in new:
        if d in old:
            file_new = os.listdir(f'openalex_new/{d}')
            with open(f'openalex_new/{d}/{file_new[0]}', 'r') as csv2_file:
                csv_reader = csv.DictReader(csv2_file)
                for row in csv_reader:
                    if row['id'] in new_ids:
                        csv_writer.writerow(row)

(Здесь я просто убирала случайно попавшие header)

In [23]:
with open('output_del.csv', 'r', newline='') as file_in, open('output_del2.csv', 'w', newline='') as file_out:
    reader = csv.DictReader(file_in)
    fieldnames = reader.fieldnames
    writer = csv.DictWriter(file_out, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        if 'title' not in row['title']:
            writer.writerow(row)

Далее я решила проанализировать аннотации: посмотреть, какие попали в удаленные, какие в новые. Заодно почистить их от слова abstract.

In [27]:
class AbstractClassifier:
    def __init__(self):
        self.should_delete = {}
        self.normal = {}

    def classify_abstracts(self, input_file):
        with open(input_file, 'r', newline='') as file_in:
            count = 0
            reader = csv.DictReader(file_in)
            for row in reader:
                abstract = row['abstract']
                if abstract == "NA":
                    count+=1 # Если содержимое отсутствует, пропускаем
                    continue
                
                try:
                    lang = detect(abstract)
                    if lang != 'en':
                        self.should_delete[row['id']] = abstract
                    else:
                        self.normal[row['id']] = abstract
                except:
                    pass
            return count

    def clean_abstracts(self):
        for id, abstract in self.normal.items():
            if abstract.startswith('Abstract'):
                cleaned_abstract = re.sub(r'^Abstract', '', abstract).strip()
                self.normal[id] = cleaned_abstract
        for id, abstract in self.should_delete.items():
            if abstract.startswith('Abstract'):
                cleaned_abstract = re.sub(r'^Abstract', '', abstract).strip()
                self.should_delete[id] = cleaned_abstract

In [28]:
classifier = AbstractClassifier()
print(classifier.classify_abstracts('output_del2.csv'))
classifier.clean_abstracts()

89365


In [25]:
print(len(classifier.should_delete))
data = list(classifier.should_delete.items())
print(data[:10])
print(len(classifier.normal))
data2 = list(classifier.normal.items())
print(data2[:10])

7317
[('https://openalex.org/W2083310319', "La notion de « verbe auxiliaire » est familière à ceux qui ont la pratique d'une langue occidentale moderne, et elle fait partie de la nomenclature grammaticale dans l'enseignement traditionnel. Mais il en est de cette notion comme de tant d'autres qu'une certaine accoutumance nous empêche de bien voir dans leur importance et leur singularité. Il s'agit d'une forme linguistique unitaire qui se réalise, à travers des paradigmes entiers, en deux éléments, dont chacun assume une partie des fonctions grammaticales, et qui sont à la fois liés et autonomes, distincts et complémentaires."), ('https://openalex.org/W2039803786', "Les caractères typologiques de la conjugaison du verbe kiranti sont examinés. Ensuite, l'auteur aborde le problème historique de la position de flexion en tibéto-birman et présente une analyse morphologique du verbe simple du lohorung, langue de l'Himalaya jadis inconnue. Enfin, le modèle du verbe proto-kiranti avancé par l'a

In [26]:
weird = {}
with open('output_del2.csv', 'r', newline='') as file:
    reader = csv.DictReader(file)
    for row in reader:
        abstract = row['abstract']
        if abstract != "NA" and row['id'] not in classifier.should_delete.keys() and row['id'] not in classifier.normal.keys():
            weird[row['id']] = abstract
print(len(weird))
lost = list(weird.items())
print(lost)

109
[('https://openalex.org/W4237108105', '-'), ('https://openalex.org/W4254983859', '-'), ('https://openalex.org/W4250484602', ''), ('https://openalex.org/W4311232101', ''), ('https://openalex.org/W4311232371', ''), ('https://openalex.org/W2026502960', ''), ('https://openalex.org/W4238769462', ''), ('https://openalex.org/W2113080297', ''), ('https://openalex.org/W2321721422', '.'), ('https://openalex.org/W2529079342', '.'), ('https://openalex.org/W2128811406', ''), ('https://openalex.org/W2343468551', ''), ('https://openalex.org/W2080340672', '.'), ('https://openalex.org/W4239614751', ''), ('https://openalex.org/W4242575532', ''), ('https://openalex.org/W2906697591', ''), ('https://openalex.org/W4252515480', ''), ('https://openalex.org/W4231713984', ''), ('https://openalex.org/W4248091307', ''), ('https://openalex.org/W2001038922', '??? ??? ???'), ('https://openalex.org/W1510704614', ''), ('https://openalex.org/W1500378219', ''), ('https://openalex.org/W1501222460', ''), ('https://ope

130694 - всего удаленных в файле\
7317 - на иностранных языках \
33898 - на английском нормальных \
89365 - NA \
109 - пустые, точка, вопросительные знаки \
5 потеряшек

In [29]:
classifier2 = AbstractClassifier()
print(classifier2.classify_abstracts('output_new.csv'))
classifier2.clean_abstracts()

2249


In [30]:
print(len(classifier2.should_delete))
data3 = list(classifier2.should_delete.items())
print(data3[:50])
print(len(classifier2.normal))
data4 = list(classifier2.normal.items())
print(data4[:50])

1378
[('https://openalex.org/W4387531215', 'RÉSUMÉCet article s’efforce de caractériser la problématique dans laquelle s’inscrit la notion structuraliste de Gesamtbedeutung, à travers une analyse comparée de textes de Hjelmslev, Jakobson et Benveniste. Tandis que les élaborations des deux premiers sont fondamentalement comparables, l’élaboration benvenistienne est dotée d’une remarquable singularité. Cette différence, qui rompt l’unité du structuralisme européen, pourtant caractérisé par sa commune mécompréhension de la théorie saussurienne, soulève deux questions importantes et remarquablement corrélatives : celle de la signification et celle du métalangage, qui sont ensuite envisagées à la lumière de la théorisation saussurienne de la langue.MOTS-CLÉS: Valeurfonctionnementsignificationmétalangagepsychanalyse Note sur l’auteurAgrégée de lettres modernes et docteur en linguistique, Anne-Gaëlle Toutain est maître d’enseignement et de recherche habilitée à diriger des recherches en lingu

In [31]:
weird_new = {}
with open('output_new.csv', 'r', newline='') as file2:
    reader2 = csv.DictReader(file2)
    for row in reader2:
        abstract = row['abstract']
        if abstract != "NA" and row['id'] not in classifier2.should_delete.keys() and row['id'] not in classifier2.normal.keys():
            weird_new[row['id']] = abstract
print(len(weird_new))
lost_new = list(weird_new.items())
print(lost_new)

9
[('https://openalex.org/W4283790427', '.'), ('https://openalex.org/W4379800887', ''), ('https://openalex.org/W4379794489', ''), ('https://openalex.org/W4380184549', ''), ('https://openalex.org/W1775895694', ''), ('https://openalex.org/W1620015025', ''), ('https://openalex.org/W4301871406', ''), ('https://openalex.org/W2949549207', ''), ('https://openalex.org/W3035435298', '-')]


8337 - всего новых аннотаций \
2249 - NA \
1378 - на иностранных \
4701 - на английском \
9 - пустые, точки, тире